### What is asyncIO?

Okay, to start, we should flesh out the difference between **parallelism** and **concurrency**.

*Parallelism* = performing multiple things at the same time

*Concurrency* = a broader term. Refers to the ability of different parts or units of a program, algorithm, or problem to be executed out-of-order or in partial order, without affecting the final outcome.

So what are some execution models for concurrency? The most common is given by **threading**, in which multiple threads take turns executing tasks. Done well enough, this gives the *illusion* of multi-processing parallelism.

![differences](http://www.dietergalea.com/images/parallel_sequential_concurrent.jpg)

`async` is itself an entirely different model for concurrency, instead following an asynchronous model. What does it mean to be **asynchronous**? Asynchronous routines are able to “pause” while waiting on their ultimate result and let other routines run in the meantime. Note this could be done on a single thread!

Python's implementation of asynchronous IO is given by the `asyncio` package, and the keywords `async` and `await`.

At the heart of async IO are **coroutines**. A coroutine is a specialized version of a Python generator function. Let’s start with a baseline definition and then build off of it as you progress here: a coroutine is a function that can suspend its execution before reaching return, and it can indirectly pass control to another coroutine for some time.

In [1]:
import asyncio

async def count():
    print("one")
    await asyncio.sleep(1) # tells event loop this process will wait for 1 sec, move on to other tasks
    print("two")
    
async def start():
    await asyncio.gather(count(), count(), count())

loop = asyncio.get_event_loop()
loop.create_task(start()) # this is needed for jupyter. otherwise run loop.run_until_complete(start())

<Task pending coro=<start() running at <ipython-input-1-76909265fc3d>:8>>

one
one
one
two
two
two


In contrast, a synchronous piece of code will act as:

In [2]:
import time

def count():
    print("one")
    time.sleep(1)
    print("two")

def start():
    for _ in range(3):
        count()

s = time.perf_counter()
start()
elapsed = time.perf_counter() - s
print(f"executed in {elapsed:0.2f} seconds.")

one
two
one
two
one
two
executed in 3.00 seconds.


The syntax `async def` introduces either a native coroutine or an asynchronous generator. The expressions `async with` and `async for` are also valid, and you’ll see them later on.

The keyword `await` passes function control back to the event loop. (It suspends the execution of the surrounding coroutine.) If Python encounters an `await f()` expression in the scope of `g()`, this is how `await` tells the event loop, “Suspend execution of `g()` until whatever I’m waiting on—the result of `f()`—is returned. In the meantime, go let something else run.”

In [3]:
async def g():
    r = await f() # pause and wait for f() to complete before coming back
    return r

In [4]:
import random

# ANSI colors
c = (
    "\033[0m",   # End of color
    "\033[36m",  # Cyan
    "\033[91m",  # Red
    "\033[35m",  # Magenta
)

async def randint(a: int, b: int) -> int:
    return random.randint(a, b)

async def makerandom(idx: int, threshold: int = 6) -> int:
    print(c[idx + 1] + f"Initiated makerandom({idx}).")
    i = await randint(0, 10)
    while i <= threshold:
        print(c[idx + 1] + f"makerandom({idx}) == {i} too low; retrying.")
        await asyncio.sleep(idx + 1)
        i = await randint(0, 10)
    print(c[idx + 1] + f"---> Finished: makerandom({idx}) == {i}" + c[0])
    return i

async def start():
    res = await asyncio.gather(*(makerandom(i, 10 - i - 1) for i in range(3)))
    return res


random.seed(444)
loop.create_task(start())

<Task pending coro=<start() running at <ipython-input-4-5297979d9222>:24>>

Initiated makerandom(2).
makerandom(2) == 4 too low; retrying.
Initiated makerandom(1).
makerandom(1) == 4 too low; retrying.
Initiated makerandom(0).
makerandom(0) == 0 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(1) == 7 too low; retrying.
makerandom(0) == 4 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 8 too low; retrying.
---> Finished: makerandom(1) == 10
makerandom(0) == 7 too low; retrying.
makerandom(0) == 8 too low; retrying.
makerandom(2) == 4 too low; retrying.
makerandom(0) == 7 too low; retrying.
makerandom(0) == 1 too low; retrying.
makerandom(0) == 6 too low; retrying.
---> Finished: makerandom(2) == 9
makerandom(0) == 3 too low; retrying.
makerandom(0) == 9 too low; retrying.
makerandom(0) == 7 too low; retrying.
---> Finished: makerandom(0) == 10
